In [23]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [24]:
import re

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [25]:
def is_towel_possible(request, patterns):
    '''
    Returns True if request can be constructed from towel patterns available and False otherwise
    patterns: set(str)
    request: str
    '''

    if not request:
        return True

    for pattern in patterns:
        if request.startswith(pattern):
            smaller_towel = request[len(pattern):]
            possible = is_towel_possible(smaller_towel, patterns)
            if possible:
                return True
            #print(request, pattern, smaller_towel)


    return False

In [26]:
patterns_info, requests_info = input.split('\n\n')
patterns = set([x[0] if x[0] else x[1] for x in re.findall(r'([a-z]+),|([a-z]+)$',patterns_info)])
requests = requests_info.split('\n')

possible_count = 0
for request in requests:
    if is_towel_possible(request, patterns):
        possible_count += 1
possible_count
    

240

# Part 2

In [27]:
def is_towel_possible(request, patterns, count=0, depth=0, seen_possible={'': 1}, seen_impossible=set()):
    '''
    Returns number of possible constructions of the request (if any), given the available patterns
    patterns: set(str)
    request: str
    '''

    # if not depth:
    #     print(f'request: {request}')

    if not request:
        return count+1

    for pattern in patterns:
        if request.startswith(pattern):
            smaller_towel = request[len(pattern):]

            # Use memorisation to reduce search space
            if smaller_towel in seen_possible:
                #print('skipping early possible', smaller_towel, seen_possible[smaller_towel], seen_possible)
                count += seen_possible[smaller_towel]
                continue
            elif smaller_towel in seen_impossible:
                #print('skipping early impossible')
                continue

            unique_towels, possibles, impossibles = is_towel_possible(
                smaller_towel, patterns, depth=depth+1, seen_possible=seen_possible, seen_impossible=seen_impossible)
            
            seen_possible.update(possibles)
            seen_impossible = seen_impossible.union(impossibles)
            
            count += unique_towels

    if count:
        seen_possible[request] = count
    else:
        seen_impossible.add(request)

    return (count, seen_possible.copy(), seen_impossible)

In [28]:
patterns_info, requests_info = input.split('\n\n')
patterns = set([x[0] if x[0] else x[1] for x in re.findall(r'([a-z]+),|([a-z]+)$',patterns_info)])
requests = requests_info.split('\n')

seen_possible = {'': 1}
seen_impossible = set()

total_unique_towels = 0
for request in requests:
    unique_towels, possibles, impossibles = is_towel_possible(
        request, patterns, seen_possible=seen_possible, seen_impossible=seen_impossible)
    total_unique_towels += unique_towels
    
    seen_impossible = seen_impossible.union(impossibles)
    #print('\n\n')
    
total_unique_towels
    

848076019766013